# Amazon Product Review Data Wrangling

The purpose of this notebook is to load and clean data from Amazon.com product reviews, ultimately this data will be incorporated into a study to determine how much reviews and/or ratings affect product sales. There are two large JSON files which contain JSON lines for each review,  and product. Both files are too large to simultaneously fit into memory so only a portion of the data will be loaded. Because Amazon product descriptions are often inconsistant among the various vendors, in addition to typical data cleaning tasks (dropping NA's, etc), I'll also run a short algorithm to determine if the datasets contain possible redundant products with only slightly different names (ie "Casio men's watch GT2HF2" fitness, vs. "Casio men's watch GT2HF2")

In [1]:
#First we need to load the various packages

import pandas as pd
import numpy as np
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime
import gzip
import json
cDir=os.getcwd()
os.chdir(os.path.abspath('C:/Users/micha/Documents/Springboard/Amazon_Product_Review_Project/Data'))
import warnings
warnings.filterwarnings('ignore')

Next I need to define a couple functions to read in the data. The first function "parse" will create a generator that will yield a JSON line from the .json.gz file. The second function will return a pandas data frame with numRow rows.

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path, numRow):
  i = 0
  df = {}
  if i <= numRow: 
      for d in parse(path):
        df[i] = d
        i += 1
  return pd.DataFrame.from_dict(df, orient='index')
    


In [3]:
reviewData = getDF('Home_and_Kitchen_5.json.gz', 100000)

I don't want to have any products in the dataset that have only a handful of reviews. From working on the feature engineering component I noticed that in a subsample of ~80,000 products I needed at least 80 instances in each category. I also want to remove any products that only exist once. 

In [4]:
reviewData.shape

(6898955, 12)

In [5]:
reviewData.describe()

,overall,unixReviewTime
count,6.898955e+06,6.898955e+06
mean,4.358473e+00,1.448832e+09
std,1.131936e+00,5.673527e+07
min,1.000000e+00,9.572256e+08
25%,4.000000e+00,1.420416e+09
50%,5.000000e+00,1.457222e+09
75%,5.000000e+00,1.487635e+09
max,5.000000e+00,1.538611e+09


In [6]:
reviewData.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image'],
      dtype='object')

In [9]:
filtered = reviewData.groupby('asin')['asin'].filter(lambda x: len(x) > 50)
filteredData = reviewData[reviewData['asin'].isin(filtered)].reset_index(drop = True)

In [8]:
filteredData.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"04 29, 2018",A2J2EILT57HKWR,B00002N601,Gpaw,"Cooked a pot roast, mmmm good, nice quality","mmmm good, nice",1524960000,NaN,{'Size:': ' 6 qt'},NaN
1,5.0,True,"04 23, 2018",AZ5WZ0KJKPFLC,B00002N601,Nom De Plume,Didn't know what size to get but careful readi...,Will Do In A Pinch,1524441600,NaN,{'Size:': ' 4 qt'},NaN
2,5.0,True,"04 23, 2018",A1TINW4RXWQPNH,B00002N601,Gustavo Q Mastroianni,Worked as expected.,Five Stars,1524441600,NaN,{'Size:': ' 6 qt'},NaN
3,5.0,True,"04 22, 2018",A39YU2G9YGNXL9,B00002N601,Kelly Rae delo,"Nice pressure cooker, love it!",Five Stars,1524355200,NaN,{'Size:': ' 8 qt'},NaN
4,5.0,True,"04 17, 2018",ASN8LDFAZLX8M,B00002N601,RBH Seattle,Great for hurry-up meals,Fast Cooker!,1523923200,NaN,{'Size:': ' 6 qt'},NaN


In [10]:
filteredData.shape

(4609450, 12)

Next I'll load the product data into a pandas dataframe. Because the JSON data file is so large, the loading process so this takes a while.

In [11]:
metaData = getDF('meta_Home_and_Kitchen.json.gz', 100000)

In [12]:
metaData.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'image',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'details'],
      dtype='object')

In [13]:
def productComparison(productData):
    '''This function takes in a dataframe of Amazon data and returns a new data frame with consistent naming convention for 
    the products'''
    #I want to keep an eye on how long this funtion takes to run because I know it's going to be a little slow
    startTime = datetime.now()
    possibleMatch = []
    # Frist identify all the unique product IDs (product_parent) and unique product names (product_title)
    uniqueParent = productData['asin'].unique()
    uniqueTitle = productData['title'].unique()
    #iterate thorugh the unique product IDs to see if the associated product title matches any of the other unique titles. 
    for product in uniqueParent:
        #The titles will change as the data is refined
        #uniqueTitle = productData['product_title'].unique()
        #First get the title associated with a product ID
        try:
            prodComp = productData.loc[productData['asin'] == product, 'title'].unique()[0]
            #now iterate through the unique titles
            for compProd in uniqueTitle:
                #No need to make any changes to the dataframe if the product name is the exact same as the comparison string...
                if compProd != prodComp:
                    #determine both the set ratio and sort ratio
                    setRatio = fuzz.token_set_ratio(prodComp, compProd)
                    sortRatio = fuzz.token_sort_ratio(prodComp, compProd)
                    #If the set ratio and sort ratio both exceed some threshold, then we will update the name of the product in the dataframe
                    if setRatio > 80 and sortRatio > 80:
                        possibleMatch.append([prodComp, compProd, setRatio, sortRatio])
                        productData.loc[productData['title'] == compProd, 'title'] = prodComp
        except:
            #there will be times that all instances of a where a product title will no longer be in the dataframe (already been changed)
            continue
        #Reset the product IDs
        productData.loc[productData['title'] == prodComp, 'parent'] = product
    processTime = datetime.now()-startTime
    print(processTime)
    return productData, possibleMatch

In [15]:

#Now I can clear the memory of the full review data set.

reviewData = []

df = pd.merge(filteredData, metaData, how = 'left', on='asin')

In [16]:
df.shape

(4903240, 29)

Now that the data are merged into one dataframe, I'll clear the reviewData and metaData dataframes to free up space.

In [17]:
reviewData = []
metaData = []

In [18]:
#Drop the large columns from the DF
df1 = df.drop(['image_x', 'tech1', 'description', 'image_y', 'tech2', 'also_buy', 'feature', 'also_view', 'similar_item', 'details'], axis = 1)

Lets take a look at the finished product. 

In [17]:
df1.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,category,fit,title,brand,rank,main_cat,date,price
0,5.0,True,"04 29, 2018",A2J2EILT57HKWR,B00002N601,Gpaw,"Cooked a pot roast, mmmm good, nice quality","mmmm good, nice",1524960000,NaN,{'Size:': ' 6 qt'},"[Home & Kitchen, Kitchen & Dining, Cookware, P...",,Presto 01241 4-Quart Aluminum Pressure Cooker,Presto,"[>#16,941 in Kitchen & Dining (See Top 100 in ...",Amazon Home,"September 9, 2003",$23.19
1,5.0,True,"04 29, 2018",A2J2EILT57HKWR,B00002N601,Gpaw,"Cooked a pot roast, mmmm good, nice quality","mmmm good, nice",1524960000,NaN,{'Size:': ' 6 qt'},"[Home & Kitchen, Kitchen & Dining, Cookware, P...",,Presto 01241 4-Quart Aluminum Pressure Cooker,Presto,"[>#16,941 in Kitchen & Dining (See Top 100 in ...",Amazon Home,"September 9, 2003",$23.19
2,5.0,True,"04 23, 2018",AZ5WZ0KJKPFLC,B00002N601,Nom De Plume,Didn't know what size to get but careful readi...,Will Do In A Pinch,1524441600,NaN,{'Size:': ' 4 qt'},"[Home & Kitchen, Kitchen & Dining, Cookware, P...",,Presto 01241 4-Quart Aluminum Pressure Cooker,Presto,"[>#16,941 in Kitchen & Dining (See Top 100 in ...",Amazon Home,"September 9, 2003",$23.19
3,5.0,True,"04 23, 2018",AZ5WZ0KJKPFLC,B00002N601,Nom De Plume,Didn't know what size to get but careful readi...,Will Do In A Pinch,1524441600,NaN,{'Size:': ' 4 qt'},"[Home & Kitchen, Kitchen & Dining, Cookware, P...",,Presto 01241 4-Quart Aluminum Pressure Cooker,Presto,"[>#16,941 in Kitchen & Dining (See Top 100 in ...",Amazon Home,"September 9, 2003",$23.19
4,5.0,True,"04 23, 2018",A1TINW4RXWQPNH,B00002N601,Gustavo Q Mastroianni,Worked as expected.,Five Stars,1524441600,NaN,{'Size:': ' 6 qt'},"[Home & Kitchen, Kitchen & Dining, Cookware, P...",,Presto 01241 4-Quart Aluminum Pressure Cooker,Presto,"[>#16,941 in Kitchen & Dining (See Top 100 in ...",Amazon Home,"September 9, 2003",$23.19


In [19]:
#Clear the orginal merged dataframe form memory.
df = []

In [20]:
#Save the data as a csv
df1.to_csv('amazonReviewData5000k.csv', index = False)

Now that the data is saved, we can take a close look at the data

In [21]:
#Create a subsample of the reviewData to merge with the metadata rather than working with the entire set

reviewDataSample = df1.sample(n = 1500000)

In [22]:
reviewDataSample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500000 entries, 361207 to 4723113
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         1500000 non-null  float64
 1   verified        1500000 non-null  bool   
 2   reviewTime      1500000 non-null  object 
 3   reviewerID      1500000 non-null  object 
 4   asin            1500000 non-null  object 
 5   reviewerName    1499900 non-null  object 
 6   reviewText      1499496 non-null  object 
 7   summary         1499726 non-null  object 
 8   unixReviewTime  1500000 non-null  int64  
 9   vote            179923 non-null   object 
 10  style           1074456 non-null  object 
 11  category        1497396 non-null  object 
 12  fit             1497396 non-null  object 
 13  title           1497396 non-null  object 
 14  brand           1497396 non-null  object 
 15  rank            1497396 non-null  object 
 16  main_cat        1497396 non-nul

In [23]:
reviewDataSample.reset_index(drop=True, inplace = True)

In [24]:
#Save the sampled data set
reviewDataSample.to_csv('amazonReviewData_sample1500k.csv', index = False)

Now lets clear all the memory and reload the data.

In [25]:
df  = pd.read_csv('amazonReviewData_sample1500k.csv')

In [26]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,category,fit,title,brand,rank,main_cat,date,price
0,5.0,True,"05 27, 2016",A1RCEZ4VG7W6Q4,B0000CFN85,Mary Payne Kinnamon,Quality thick paper and worked wonders for my ...,Five Stars,1464307200,NaN,{'Color:': ' Jumbo -White'},"['Home & Kitchen', 'Kitchen & Dining', 'Bakewa...",NaN,"Wilton 415-2505 White Standard 75 Baking Cups,...",Wilton,"['>#9,714 in Kitchen & Dining (See Top 100 in ...",Amazon Home,"March 10, 2004",$3.43
1,5.0,True,"08 15, 2014",A3LTJ22VDCATD1,B001BN8Z2K,Mom of 4,Covers more area in a shorter amount of time. ...,Large head cuts down your work time - twists l...,1408060800,NaN,NaN,"['Home & Kitchen', 'Vacuums & Floor Care', 'Va...",NaN,Miele SBB 400-3 Parquet Twister XL Smooth Floo...,Miele,"['>#47,359 in Home & Kitchen (See Top 100 in H...",Amazon Home,NaN,$57.85
2,5.0,True,"12 30, 2016",A9OHV8NJ03WVH,B0026SPXRY,Susana Pachas,It's great! Easy to assemble and sturdy it's ...,It's great! Easy to assemble and sturdy it's n...,1483056000,NaN,"{'Style Name:': "" BAKER'S RACK""}","['Home & Kitchen', 'Furniture', 'Kitchen & Din...",NaN,Whitmor Supreme Sink Shelf - Multiuse Organize...,Whitmor,"['>#2,026 in Home & Kitchen (See Top 100 in Ho...",Amazon Home,NaN,$28.93
3,4.0,True,"09 28, 2015",AT3P07T5RVNTQ,B00ESAR0JW,Hstar13,We haven't had a real oven for over a year now...,"This cooks most things pretty well, but you ha...",1443398400,NaN,NaN,"['Home & Kitchen', 'Kitchen & Dining', 'Small ...",NaN,Hamilton Beach 31103A Countertop Oven with Con...,Hamilton Beach,"['>#6,672 in Kitchen & Dining (See Top 100 in ...",Amazon Home,"August 9, 2013",$7.05
4,3.0,True,"06 23, 2017",A4I78M7IQPW12,B0074V10JS,Pamela,Nice bedskirt but won't work with a bed that h...,Nice bedskirt,1498176000,NaN,"{'Size:': ' Queen/King', 'Color:': ' Burgundy'}","['Home & Kitchen', 'Bedding', 'Bed Skirts']",NaN,Hdetails Bed Skirts/Elastic Ruffles - Bedding ...,Hdetails,"['>#76,184 in Home & Kitchen (See Top 100 in H...",Amazon Home,NaN,$12.99


Several of these columns may be unnecessary. I'm going to explore how many of these columns have a majority of NaNs. 

In [27]:
df.isnull().sum()

overall                 0
verified                0
reviewTime              0
reviewerID              0
asin                    0
reviewerName          193
reviewText            504
summary               283
unixReviewTime          0
vote              1320077
style              425544
category             2604
fit               1499553
title                2604
brand               10268
rank                 2604
main_cat             4082
date               664029
price              263237
dtype: int64

Since style, fit, and date may not be relevant for analyzing the affect of reviews or overall ratings on purchases, these columns will be dropped. Additionlly, 'vote' appears to be almost entirely missing and the 'overall' column is the rating anyway so 'vote' will also be dropped. With the exception of price, the remaining missing values are small enough that I will happliy drop the rows. It's unfortuntate how many missing values there are for price, because it would make sense for a customer to pay more attention to reviews as price increases. I may need to circle back to try to find a supplemental sample of review to substitute the rows that I will need to drop here, because it is likely not reasonable to impute these with the mean given the range of values. 

In [28]:
df = df.drop(['vote', 'style', 'fit', 'date'], axis = 1)
df = df.dropna(axis=0, how = 'any')

In [29]:
#Lets take a look at the data now

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1231100 entries, 0 to 1499999
Data columns (total 15 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         1231100 non-null  float64
 1   verified        1231100 non-null  bool   
 2   reviewTime      1231100 non-null  object 
 3   reviewerID      1231100 non-null  object 
 4   asin            1231100 non-null  object 
 5   reviewerName    1231100 non-null  object 
 6   reviewText      1231100 non-null  object 
 7   summary         1231100 non-null  object 
 8   unixReviewTime  1231100 non-null  int64  
 9   category        1231100 non-null  object 
 10  title           1231100 non-null  object 
 11  brand           1231100 non-null  object 
 12  rank            1231100 non-null  object 
 13  main_cat        1231100 non-null  object 
 14  price           1231100 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(12)
memory usage: 142.1+ MB


Looks better, but we need price to be a float, and rank to be an int. Updating rank will require some string manipulation to extract the rank within the specific subcategory of Home & Kitchen (i.e. Laundry Bags)

In [30]:
#First parse the rank on commas and store the result in a new column: rankCat
df['rankCat'] = df['rank'].str.split('>#', n = -1, expand = False)


In [31]:
len(df['category'].unique())

1285

In [32]:
#Check for duplicate products
checkedData, possibleMatches = productComparison(df)

11:55:58.574257


In [45]:
possibleMatches[0]


['Hamilton Beach 31103A Countertop Oven with Convection and Rotisserie',
 'Hamilton Beach Countertop Oven with Convection and Rotisserie',
 100,
 95]

In [46]:
len(possibleMatches)

6721

In [34]:
df = checkedData

Now we need to get the ranking of the products for their specific category. This is the second ranking in the list created when we parsed the category ranking. The first element of the list is generally a bracket. We'll standardize the elements using a couple lambda functions. 

In [35]:
df['rankCat'] = df['rankCat'].apply(lambda x: x[1::] if x[0] == "['" else x)
df['rankCat'] = df['rankCat'].apply(lambda x: x[1::] if x[0] == "[" else x)

Now we use another lambda function to grab the ranking for the specific category and store it in the dataframe. 

In [36]:
df['catRank'] = df['rankCat'].apply(lambda x: x[1] if len(x) >= 2 else x[0])


Now we need to grab the first element of the category rank and convert it to an interger
        

In [37]:
df['catRank'] = df['catRank'].apply(lambda x: x[1::] if x[0] == "[" else x)
df['Ranking'] = df['catRank'].apply(lambda x: x.split()[0])
#Now we should have the rankings but before we convert them to ints we need to drop commas
df['Ranking'] = df['Ranking'].str.replace(',', '')
df['Ranking'] = df['Ranking'].apply(lambda x: int(x) if x.isdigit() else np.nan)
#Now we need to drop any products that don't have rankings.
df = df.dropna(axis=0, how = 'any')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1219044 entries, 0 to 1499999
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         1219044 non-null  float64
 1   verified        1219044 non-null  bool   
 2   reviewTime      1219044 non-null  object 
 3   reviewerID      1219044 non-null  object 
 4   asin            1219044 non-null  object 
 5   reviewerName    1219044 non-null  object 
 6   reviewText      1219044 non-null  object 
 7   summary         1219044 non-null  object 
 8   unixReviewTime  1219044 non-null  int64  
 9   category        1219044 non-null  object 
 10  title           1219044 non-null  object 
 11  brand           1219044 non-null  object 
 12  rank            1219044 non-null  object 
 13  main_cat        1219044 non-null  object 
 14  price           1219044 non-null  object 
 15  rankCat         1219044 non-null  object 
 16  parent          1219044 non-null  ob

Now I need to convert the price from $xxx.xx to a float of the form xxx.xx

In [39]:
df['price'] = df['price'].apply(lambda x: str(x))

In [40]:
df['price'] = df['price'].str.replace('$', '')
df['price'] = df['price'].str.replace(',', '')
#There are some errors in the price data. It seems there are some product descriptions that popped in somehow. To resolve this
#I'll replace any long values with 0 so we can drop them later. Anything over 10000.00 will be dropped
df['price'] = df['price'].apply(lambda x: str(0) if len(x) > 8 else x)
df['price'] = df['price'].apply(lambda x: float(x))

In [41]:
#Now we need to drop any products that don't have prices].
df = df.dropna(axis=0, how = 'any')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1219044 entries, 0 to 1499999
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         1219044 non-null  float64
 1   verified        1219044 non-null  bool   
 2   reviewTime      1219044 non-null  object 
 3   reviewerID      1219044 non-null  object 
 4   asin            1219044 non-null  object 
 5   reviewerName    1219044 non-null  object 
 6   reviewText      1219044 non-null  object 
 7   summary         1219044 non-null  object 
 8   unixReviewTime  1219044 non-null  int64  
 9   category        1219044 non-null  object 
 10  title           1219044 non-null  object 
 11  brand           1219044 non-null  object 
 12  rank            1219044 non-null  object 
 13  main_cat        1219044 non-null  object 
 14  price           1219044 non-null  float64
 15  rankCat         1219044 non-null  object 
 16  parent          1219044 non-null  ob

In [43]:
#Now that the data is cleaned I want to save it again.

df.to_csv('AmazonDataCleaned1500k.csv', index = False)